# More examples

In [1]:
# showing entire fault system participation rates

In [2]:
#import pathlib
import json
import nzshm_model as nm
import geopandas as gpd
import solvis

# from solvis.fault_system_solution_helper import FaultSystemSolutionHelper
from solvis_graphql_api.color_scale import ColourScaleNormaliseEnum, get_colour_values, get_colour_scale
from ipyleaflet import Map, GeoJSON, LegendControl, FullScreenControl, Popup, ScaleControl, WidgetControl
from ipywidgets import HTML

warning openquake module dependency not available, maybe you want to install
                with nzshm-model[openquake]


Running without `toshi` options


In [3]:
# OPTIONS
SINGLE_SOLUTION = False
SECTION_RATE = False # otherwise Parent rates

if SINGLE_SOLUTION:
    solution = solvis.InversionSolution.from_archive("NZSHM22_ScaledInversionSolution-QXV0b21hdGlvblRhc2s6MTEzMTM0.zip")
else:
    current_model = nm.get_model_version("NSHM_v1.0.4")
    slt = current_model.source_logic_tree
    csol = solvis.CompositeSolution.from_archive("NSHM_v1.0.4_CompositeSolution.zip", slt)
    solution = csol._solutions['CRU']
# 
# helper = FaultSystemSolutionHelper(solution)  

## Choose some faults and get their unique rupture IDs

### A) Calculate participation rates

In [4]:
rate_column = "Annual Rate" if isinstance(solution, solvis.InversionSolution) else "rate_weighted_mean"

rs_df = solution.rs_with_rupture_rates
if not SECTION_RATE:
    pf_df = rs_df.join(solution.fault_sections[['ParentID']], on='section')
    fault_rates = pf_df[["Rupture Index", "ParentID", rate_column]]
    parent_rates = fault_rates.groupby("ParentID").agg('sum')[rate_column]
    parent_rates.head()

In [5]:
# let's get the participation rate for each subsection in the rupture
if SECTION_RATE:
    section_rates = rs_df[["Rupture Index", "section", rate_column]].groupby("section").agg('sum')[rate_column]
    section_rates.head()

In [6]:
# # check sum of rates matches
# if not parent_rates.sum() == section_rates.sum():
#     print(parent_rates.sum())
#     print(section_rates.sum())

### Style the geojson using a color scale

In [7]:
surfaces = solution.fault_surfaces()
# surfaces = surfaces[surfaces["FaultID"].isin(subsections)]
if SECTION_RATE:
    rate_series = section_rates
    surfaces = surfaces.join(rate_series, on='FaultID', how='outer')
else:
    rate_series = parent_rates 
    surfaces = surfaces.join(rate_series, on='ParentID', how='outer')
surfaces.rename(columns={rate_series.name: "annual_rate"}, inplace=True)

fault_sections_gdf = gpd.GeoDataFrame(surfaces)

rate_series = section_rates if SECTION_RATE else parent_rates
print('min', rate_series.min())
# create the colour scale from our list of parent_fault_rates
color_values = get_colour_values(
                color_scale="inferno",
                color_scale_vmax=rate_series.max(),
                color_scale_vmin=max(rate_series.min(), 1e-20),
                color_scale_normalise= ColourScaleNormaliseEnum.LOG.value,
                values=tuple(rate_series.tolist()),
            )

rate_ids = rate_series.index.tolist()
data = json.loads(fault_sections_gdf.to_json())

# # merge the styling with the geojson
for feature in data["features"]:
    try:
        if SECTION_RATE:
            color = color_values[rate_ids.index(feature['properties']['FaultID'])]
        else:
            color = color_values[rate_ids.index(feature['properties']['ParentID'])]
    except (ValueError):
        print(f"warning no rate found for index: {feature['properties']['FaultID']} faultname: `{feature['properties']['FaultName']}`")
        color = 'cyan'
    feature["properties"]["style"] = {
        "color": "#000000",
        "weight": 1,
        "fillColor": color,
        "fillOpacity": 1,
    }

min 6.19915589404e-07
warning no rate found for index: 105 faultname: `Ararata, Subsection 0`
warning no rate found for index: 106 faultname: `Ararata, Subsection 1`
warning no rate found for index: 170 faultname: `Bidwill, Subsection 0`
warning no rate found for index: 171 faultname: `Bidwill, Subsection 1`
warning no rate found for index: 405 faultname: `Central Balleny, Subsection 0`
warning no rate found for index: 406 faultname: `Central Balleny, Subsection 1`
warning no rate found for index: 407 faultname: `Central Balleny, Subsection 2`
warning no rate found for index: 408 faultname: `Central Balleny, Subsection 3`
warning no rate found for index: 409 faultname: `Central Balleny, Subsection 4`
warning no rate found for index: 410 faultname: `Central Balleny, Subsection 5`
warning no rate found for index: 501 faultname: `Dry River - Huangarua: 2, Subsection 0`
warning no rate found for index: 502 faultname: `Dry River - Huangarua: 2, Subsection 1`
warning no rate found for index:

In [8]:
# from SGI
cs = get_colour_scale(
        color_scale="inferno",
        color_scale_normalise=ColourScaleNormaliseEnum.LOG.value, 
        vmax=rate_series.max(),
        vmin=max(rate_series.min(), 1e-20))

color_map = dict(zip(reversed(cs.color_map.levels), reversed(cs.color_map.hexrgbs)))


### Display with ipyleaflet

In [9]:
center = [-41.5, 175]
zoom = 7
map = Map(center=center, zoom=zoom)

section_info = HTML()
section_info.value = "<b>Section Detail</b><br/><p>hover over fault sections for more details.</p>"
widget_control = WidgetControl(widget=section_info, position='topright')
legend = LegendControl(color_map, title="Rupture Rate/yr", position="topright")

def on_hover_callback(event, feature, properties, id):
    section_info.value = f"<b>{properties['FaultName']}</b>"
    section_info.value += "<br />"
    section_info.value += f"Dip: {properties['DipDeg']}</br>"
    section_info.value += f"Rake: {properties['Rake']}</br>"
    section_info.value += f"Lower depth: {round(properties['LowDepth'],3)}</br>"    
    section_info.value += f"Participation rate: {properties['annual_rate']:.2E}</br>" 
    
g = GeoJSON(data=data, 
            hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 0.1})
g.on_hover(on_hover_callback)

map.add(g)
map.add(FullScreenControl())
map.add(widget_control)
map.add(legend)
map.add(ScaleControl(position='bottomleft', max_width=250))

#display time ... note the fullscreen option 
map

Map(center=[-41.5, 175], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [10]:
# section_aggregates_detail[['section_participation_rate']]